In [13]:
import sys
print(sys.executable)

/workspace/GPT/.venv/bin/python


In [14]:
!uv pip uninstall ipywidgets && pip install ipywidgets
!uv pip show jupyter ipywidgets

Using Python 3.10.16 environment at: /workspace/GPT/.venv
Using Python 3.10.16 environment at: /workspace/GPT/.venv
Name: jupyter
Version: 1.1.1
Location: /workspace/GPT/.venv/lib/python3.10/site-packages
Requires: ipykernel, ipywidgets, jupyter-console, jupyterlab, nbconvert, notebook
Required-by:


In [15]:
import sys
import os

# Get the parent directory of the current notebook
sys.path.append(os.path.abspath(".."))

In [16]:
import torch
import multiprocessing

from data import get_wikitext_data, get_tiktoken_tokenizer, tiktoken_tokenize_dataset, save_data, load_data

In [17]:
batch_size = 64  # Kept the same; could be adjusted based on hardware
block_size = 1024  # GPT-2 uses a context length of 1024 tokens
max_iters = 50000  # More iterations needed for larger models
eval_interval = 1000  # Increase since more iterations are done
learning_rate = 5e-5  # GPT-2 uses a lower learning rate
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 500  # More frequent evaluation for stability

n_embd = 768  # GPT-2 uses 768 for the small version, 1024 for medium, 1280 for large, 1600 for XL
n_head = 12  # GPT-2 uses 12 attention heads
n_layer = 12  # GPT-2 has 12 transformer blocks in the small version
dropout = 0.1  # GPT-2 uses 0.1 dropout for better generalization
print(device)

cpu


In [18]:
DATA_PATH = "/workspace/GPT/data/tiktoken_tokenized_wikitext"
num_cores = multiprocessing.cpu_count()

tokenizer = get_tiktoken_tokenizer()
dataset = get_wikitext_data()

In [19]:
tokenized_dataset = tiktoken_tokenize_dataset(dataset, tokenizer, num_cores)
print(tokenized_dataset)

def convert_to_tensor(example):
    return {"tokens": torch.tensor(example["input_ids"], dtype=torch.long)}

tokenized_dataset = tokenized_dataset.map(convert_to_tensor, batched=False)

print(tokenized_dataset)
save_data(tokenized_dataset, DATA_PATH)

TypeError: DatasetDict.map() got an unexpected keyword argument 'disable_tqdm'

In [ ]:
tokenized_dataset = load_data(DATA_PATH)
print("DATA LOADED IN")

In [ ]:
def get_batch(split):
    data = dataset[split]["tokens"]
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

X, Y = get_batch("train")
print(X)
print(Y)